In [58]:
from gensim.models import KeyedVectors,word2vec

#model=KeyedVectors.load_word2vec_format("C:\Users\hedi\Desktop\factory619 internship\GoogleNews-vectors-negative300.bin", binary=True)

word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, limit=5000000)
type

In [59]:
import logging
import pandas as pd
import numpy as np
from numpy import random
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline
type(word_vectors)

gensim.models.keyedvectors.Word2VecKeyedVectors

In [3]:
import re
def importtxtfile(path):
    list_of_lists = []
    with open(path) as f:
        for line in f:
            z=re.split('; | , |\*|\n',line)
 #         inner_list = [line.strip() for line in re.split(',|.', line)]
            for i in z :
                list_of_lists.append( i)
    return (list_of_lists)


In [4]:
dataobj=importtxtfile("C:/Users/hedi/Desktop/factory619 internship/subjectivity_classifier-master/data/subj_dataset/objective_cv_train.txt")
datasub=importtxtfile("C:/Users/hedi/Desktop/factory619 internship/subjectivity_classifier-master/data/subj_dataset/subjective_cv_train.txt")

In [5]:
def cleaningdata(list):
    for i in list:
        if (len(i)<2) :
            list.remove(i)
    return(list)



In [6]:
dataobj=cleaningdata(dataobj)
datasub=cleaningdata(datasub)
# x=list(zip(obj, sub))
# type(x)

In [9]:
df1=pd.DataFrame({"article":dataobj})
df2=pd.DataFrame({"article":datasub})
df1["subj or obj"]= 0
df2["subj or obj"]= 1
df = pd.concat([df1, df2], ignore_index=True)


In [8]:
len(dataobj)




10629

In [11]:
df.sample(frac=1).reset_index(drop=True)

,article,subj or obj
0,anita is 14,0
1,evokes a palpable sense of disconnection,1
2,and all he does is milk it with despondent eye...,1
3,and maybe even a notch above the previous runn...,1
4,he falls in love with a village girl,0
...,...,...
19867,and if it isn't entirely persuasive,1
19868,after an attempt is made on his life by unknow...,0
19869,but how can graham really fall in love with he...,0
19870,indifferent to all except her career,0


In [12]:
my_tags = [1,0]
plt.figure(figsize=(10,4))
df.sub or obj.value_counts().plot(kind='bar');


<Figure size 720x288 with 0 Axes>

In [13]:
import nltk


def print_plot(index):
    example = df[df.index == index][['article', 'subj or obj']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Tag:', example[1])
print_plot(10)


nltk.download('stopwords')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df['article'] = df['article'].apply(clean_text)
print_plot(10)

she
Tag: 0


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hedi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\hedi\.conda\envs\factory619\lib\site-packages\bs4\__init__.py:294: UserWarning: "b' .'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)



Tag: 0


In [99]:
df.sample(frac=1).reset_index(drop=True)

df.to_csv("suborobj.csv")

In [14]:
X = df.article
y = df["subj or obj"] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [15]:

X_train.head()


16619     may dramatic gutwrenching impact holocaust films
9507                                               dynamic
16588    moores better fingering problems finding solut...
2536     thomas narrowly escapes brother sean finds wis...
9189                  festered poisonous snakes alligators
Name: article, dtype: object

In [76]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

# %%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.8262328077826233
              precision    recall  f1-score   support

           0       0.82      0.87      0.84      3198
           1       0.84      0.78      0.81      2764

    accuracy                           0.83      5962
   macro avg       0.83      0.82      0.82      5962
weighted avg       0.83      0.83      0.83      5962



In [17]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)


y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.6766185843676619
              precision    recall  f1-score   support

           0       0.63      0.97      0.76      3198
           1       0.90      0.34      0.50      2764

    accuracy                           0.68      5962
   macro avg       0.76      0.65      0.63      5962
weighted avg       0.75      0.68      0.64      5962



In [18]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)


y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

C:\Users\hedi\.conda\envs\factory619\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


accuracy 0.7512579671251258
              precision    recall  f1-score   support

           0       0.76      0.79      0.77      3198
           1       0.75      0.70      0.72      2764

    accuracy                           0.75      5962
   macro avg       0.75      0.75      0.75      5962
weighted avg       0.75      0.75      0.75      5962



In [60]:
from gensim.models import KeyedVectors,word2vec

#model=KeyedVectors.load_word2vec_format("C:\Users\hedi\Desktop\factory619 internship\GoogleNews-vectors-negative300.bin", binary=True)

wv = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, limit=500000)
wv.init_sims(replace=True)
from itertools import islice
list(islice(wv.vocab, 13030, 13050))
type(wv)

gensim.models.keyedvectors.Word2VecKeyedVectors

In [20]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [63]:

    
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens
    
train, test = train_test_split(df, test_size=0.3, random_state = 42)

test_tokenized = test.apply(lambda r: w2v_tokenize_text(r['article']), axis=1).values
train_tokenized = train.apply(lambda r: w2v_tokenize_text(r['article']), axis=1).values

X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)






# def w2v_tokenize_text(text):
#     tokens = []
#     for sent in nltk.sent_tokenize(text, language='english'):
#         for word in nltk.word_tokenize(sent, language='english'):
#             if (len(word)<2):
#                 continue
#             tokens.append(word)
#     return tokens
    
# train, test = train_test_split(df, test_size=0.3, random_state = 42)

# test_tokenized = test.apply(lambda r: w2v_tokenize_text(r['article']), axis=1).values
# train_tokenized = train.apply(lambda r: w2v_tokenize_text(r['article']), axis=1).values

# X_train_word_average = word_averaging_list(wv,train_tokenized)
# X_test_word_average = word_averaging_list(wv,test_tokenized)

C:\Users\hedi\.conda\envs\factory619\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  


In [24]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_word_average, train["subj or obj"])
y_pred = logreg.predict(X_test_word_average)
print('accuracy %s' % accuracy_score(y_pred, test["subj or obj"]))
print(classification_report(test["subj or obj"], y_pred))

C:\Users\hedi\.conda\envs\factory619\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


accuracy 0.8186850050318685
              precision    recall  f1-score   support

           0       0.82      0.84      0.83      3198
           1       0.81      0.79      0.80      2764

    accuracy                           0.82      5962
   macro avg       0.82      0.82      0.82      5962
weighted avg       0.82      0.82      0.82      5962



In [26]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [39]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,max_depth = 5, alpha = 10, n_estimators = 300)


In [1]:
from xgboost import XGBClassifier
# # train XGBoost model with default parameters
my_model = XGBClassifier()
model = XGBClassifier()
my_model.fit(X_train_word_average, train["class"] ,verbose=False)



               
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# print('accuracy %s' % accuracy_score(y_pred, test["subj or obj"]))
# # and save our model
# my_model.save_model("xgboost_baseline.model")

NameError: name 'X_train_word_average' is not defined

In [41]:
y_pred = my_model.predict(X_test_word_average)
print('accuracy %s' % accuracy_score(y_pred, test["subj or obj"]))

accuracy 0.8151626970815162


In [37]:
#plt.figure(figsize=(25,8))sns.scatterplot(x=pca_vectors[:, 0], y=pca_vectors[:, 1], hue=label_df)plt.title('Principal Components vs Class distribution', fontsize=16)plt.ylabel('Principal Component 2', fontsize=16)plt.xlabel('Principal Component 1', fontsize=16)plt.xticks(rotation='vertical');

In [49]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier

etree_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200))])
etree_w2v_tfidf = Pipeline([
    ("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200))])

In [50]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.itervalues().next())

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [74]:
# import gensim
# # let X be a list of tokenized texts (i.e. list of lists of tokens)
# model = gensim.models.Word2Vec(X, size=100)
# w2v = dict(zip(model.wv.index2word, model.wv.syn0))



# from sklearn.pipeline import Pipeline
# from sklearn.ensemble import ExtraTreesClassifier

# # etree_w2v = Pipeline([
# #     ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),
# #     ("extra trees", ExtraTreesClassifier(n_estimators=200))])
# etree_w2v_tfidf = Pipeline([
#     ("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)),
#     ("extra trees", ExtraTreesClassifier(n_estimators=200))])

In [71]:
%matplotlib
from jyquickhelper import add_notebook_menu
add_notebook_menu("begin","end")

Using matplotlib backend: Qt5Agg


'<div id="begin">run previous cell, wait for 2 seconds</div>\n<script>\nfunction repeat_indent_string(n){\n    var a = "" ;\n    for ( ; n > 0 ; --n)\n        a += "    ";\n    return a;\n}\n// look up into all sections and builds an automated menu //\nvar update_menu_string = function(begin, lfirst, llast, sformat, send, keep_item, begin_format, end_format) {\n    var anchors = document.getElementsByClassName("section");\n    if (anchors.length == 0) {\n        anchors = document.getElementsByClassName("text_cell_render rendered_html");\n    }\n    var i,t;\n    var text_menu = begin;\n    var text_memo = "<pre>\\nlength:" + anchors.length + "\\n";\n    var ind = "";\n    var memo_level = 1;\n    var href;\n    var tags = [];\n    var main_item = 0;\n    var format_open = 0;\n    for (i = 0; i <= llast; i++)\n        tags.push("h" + i);\n\n    for (i = 0; i < anchors.length; i++) {\n        text_memo += "**" + anchors[i].id + "--\\n";\n\n        var child = null;\n        for(t = 0; t

In [103]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

classifier1= Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
#                 ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])


classifier2=Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
#                  ('ctf', LogisticRegression()),
               ])

classifier = Pipeline([
    
    ('features', FeatureUnion([
         ('text',classifier1),
         ('words',classifier2),
   ])),
    ('clf', XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1))
#     ('clf', RandomForestClassifier()),
    ])

In [96]:
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, test["subj or obj"]))

accuracy 0.7086548138208655


In [108]:
import xgboost as xgb
train= xgb.DMatrix(X_train_word_average, label=y_train)
test= xgb.DMatrix(X_test_word_average, label=y_test)

In [117]:
param={'max_depth':3,}
model=xgb.train(param,train)
y_pred=model.predict(test)
y_pred


array([0.10336033, 0.05533749, 0.69092953, ..., 0.51425004, 0.55481684,
       0.58319634], dtype=float32)

In [28]:
import newspaper

site = newspaper.build('https://babylonbee.com/')

# for article in cnn_paper.articles:
#     print(article.url)

In [29]:
list1=[]
type(list1)

list

In [30]:
site = newspaper.build('https://babylonbee.com/')
list1=[]
for article in site.articles :
    article.download()
    article.parse()
    article.nlp()
    art=article.text
    auth=article.authors
    date=article.publish_date
    summary=article.summary
    title=article.title
    dec={"auth": auth,"date" : date,"summary" : summary,"title": title,"art" : art}
    list1.append(dec)


In [31]:
list1


[{'auth': [],
  'date': None,
  'summary': '',
  'title': 'Your Trusted Source for Christian News Satire.',
  'art': ''},
 {'auth': [],
  'date': None,
  'summary': 'PoliticsEl Paso Man Comes Down From Insane Acid Trip Where He Hallucinated That He Ran For President',
  'title': 'Your Trusted Source for Christian News Satire.',
  'art': 'Politics\n\n\n\nEl Paso Man Comes Down From Insane Acid Trip Where He Hallucinated That He Ran For President'},
 {'auth': [],
  'date': None,
  'summary': 'PoliticsEl Paso Man Comes Down From Insane Acid Trip Where He Hallucinated That He Ran For President',
  'title': 'Your Trusted Source for Christian News Satire.',
  'art': 'Politics\n\n\n\nEl Paso Man Comes Down From Insane Acid Trip Where He Hallucinated That He Ran For President'},
 {'auth': [],
  'date': None,
  'summary': 'PoliticsEl Paso Man Comes Down From Insane Acid Trip Where He Hallucinated That He Ran For President',
  'title': 'Your Trusted Source for Christian News Satire.',
  'art': '